In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

In [3]:
from qml_hep_lhc.ansatzes import Farhi
import cirq
import numpy as np
import tensorflow_quantum as tfq
import tensorflow as tf
import matplotlib.pyplot as plt

In [30]:
depth = 2
n = 4
ansatz = Farhi()
qc,_, symbols, op = ansatz.build(cirq.GridQubit.rect(1,n),None,depth, False)
num_symb = len(symbols)

expectation = tfq.layers.Expectation()

In [31]:
qc

(-1, -1): ───X───H───XX────────XX────────XX────────XX────────ZZ────────ZZ────────ZZ────────ZZ────────H───
                     │         │         │         │         │         │         │         │
(0, 0): ─────────────XX^(θ0)───┼─────────┼─────────┼─────────ZZ^(θ1)───┼─────────┼─────────┼─────────────
                               │         │         │                   │         │         │
(0, 1): ───────────────────────XX^(θ2)───┼─────────┼───────────────────ZZ^(θ3)───┼─────────┼─────────────
                                         │         │                             │         │
(0, 2): ─────────────────────────────────XX^(θ4)───┼─────────────────────────────ZZ^(θ5)───┼─────────────
                                                   │                                       │
(0, 3): ───────────────────────────────────────────XX^(θ6)─────────────────────────────────ZZ^(θ7)───────

In [32]:
mse = tf.keras.losses.MeanSquaredError()

In [41]:
def cost(rotations):
    circuit_tensor = tfq.convert_to_tensor([qc])
    rotations = tf.convert_to_tensor([rotations])
    res = expectation(circuit_tensor,
                      operators=op,
                      symbol_names=symbols,
                      symbol_values=rotations)
    return res.numpy()[0]

def generate_surface(cost_function):
    Z = []
    Z_assembler = []

    X = np.arange(-np.pi, np.pi, 0.25)
    Y = np.arange(-np.pi, np.pi, 0.25)
    
    X, Y = np.meshgrid(X, Y)
    
    for x in X[0, :]:
        for y in Y[:, 0]:
            rotations = [x]*(num_symb//2) + [y]*(num_symb - (num_symb//2))
            Z_assembler.append(cost_function(rotations))
        Z.append(Z_assembler)
        Z_assembler = []

    Z = np.asarray(Z)
    return Z

def plot_surface(surface):
    X = np.arange(-np.pi, np.pi, 0.25)
    Y = np.arange(-np.pi, np.pi, 0.25)
    X, Y = np.meshgrid(X, Y)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    surf = ax.plot_surface(X, Y, surface, cmap="viridis", linewidth=0, antialiased=False)
    ax.set_zlim(0, 1)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter("%.02f"))
    plt.show()

In [42]:
global_surface = generate_surface(cost)

In [ ]:
plot_surface(global_surface)

In [44]:
global_surface

array([[[0.47454557],
        [0.970137  ],
        [0.6285902 ],
        [0.57787997],
        [0.9663036 ],
        [0.55393165],
        [0.39417356],
        [0.43038526],
        [0.47454602],
        [0.97013676],
        [0.62858975],
        [0.57788026],
        [0.96630377],
        [0.5539316 ],
        [0.39417368],
        [0.4303853 ],
        [0.474546  ],
        [0.970137  ],
        [0.6285895 ],
        [0.5778803 ],
        [0.9663039 ],
        [0.5539318 ],
        [0.39417356],
        [0.4303853 ],
        [0.47454628],
        [0.970137  ]],

       [[0.9701369 ],
        [0.6353691 ],
        [0.41432956],
        [0.44249603],
        [0.5539323 ],
        [0.9876288 ],
        [0.6127305 ],
        [0.5673299 ],
        [0.9701371 ],
        [0.63536894],
        [0.4143298 ],
        [0.44249582],
        [0.5539326 ],
        [0.9876288 ],
        [0.6127304 ],
        [0.5673298 ],
        [0.9701372 ],
        [0.63536865],
        [0.4143299 ],
        